# Day Two: Account Enrichment

This notebook enriches BBF Account records with additional fields from ES Account.

## How It Works
1. Reads field mappings from `mappings/ES_Account_to_BBF_Account_mapping.xlsx`
2. Uses the Excel to determine which BBF fields map to which ES fields
3. Uses picklist translations from the mapping file
4. Queries migrated records (linked via ES_Legacy_ID__c / BBF_New_Id__c)
5. Enriches BBF fields that are empty but have ES source data

## Day 1 Fields (Already Migrated - Excluded)
- Name, BillingStreet, BillingCity, BillingState, BillingPostalCode
- Phone, Website, Industry, Type
- OwnerId, ES_Legacy_ID__c

In [25]:
# === SETUP & IMPORTS ===

import sys
import os
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment
from datetime import datetime

# Import the mapping reader
from mapping_reader import (
    load_mapping,
    get_enrichment_fields,
    translate_picklist,
    print_mapping_summary,
)

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("\n✅ Setup complete")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3

✅ Setup complete


In [26]:
# === LOAD FIELD MAPPING FROM EXCEL ===

print("=" * 80)
print("LOADING FIELD MAPPING FROM EXCEL")
print("=" * 80)

MAPPING_FILE = "ES_Account_to_BBF_Account_mapping.xlsx"

# Load the mapping
mapping = load_mapping(MAPPING_FILE)

# Print summary
print_mapping_summary(mapping)

# Day 1 fields that were already migrated (exclude from enrichment)
DAY1_FIELDS = [
    "Name",
    "BillingStreet",
    "BillingCity",
    "BillingState",
    "BillingPostalCode",
    "Phone",
    "Website",
    "Industry",
    "Type",
    "OwnerId",
    "ES_Legacy_ID__c",
]

# Get enrichment fields (excludes Day 1 and system fields)
ENRICHMENT_MAPPING = get_enrichment_fields(mapping, DAY1_FIELDS)

print(f"\n📋 Fields to enrich (from Excel mapping):")
for bbf_field, es_field in ENRICHMENT_MAPPING.items():
    print(f"   {bbf_field:<35} <- {es_field}")

print(f"\nTotal enrichment fields: {len(ENRICHMENT_MAPPING)}")

LOADING FIELD MAPPING FROM EXCEL

Mapping: ES_Account_to_BBF_Account_mapping.xlsx
Total field mappings: 61
Enrichable fields: 61
Picklist fields with translations: 13

Enrichable Fields:
  Type                                <- Type                           (High)
  ParentId                            <- ParentId                       (High)
  BillingStreet                       <- BillingStreet                  (High)
  BillingCity                         <- BillingCity                    (High)
  BillingState                        <- BillingState                   (High)
  BillingPostalCode                   <- BillingPostalCode              (High)
  BillingCountry                      <- BillingCountry                 (High)
  BillingLatitude                     <- BillingLatitude                (High)
  BillingLongitude                    <- BillingLongitude               (High)
  BillingGeocodeAccuracy              <- BillingGeocodeAccuracy         (High)
  ... and 51 more

Pick

In [27]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# BBF Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"

# Enrichment Options
DRY_RUN = True  # Set to False to actually update records
TEST_LIMIT = None  # Set to None for all records

# Output file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"account_enrichment_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   DRY_RUN: {DRY_RUN}")
print(f"   TEST_LIMIT: {TEST_LIMIT}")
print(f"   Output: {output_file}")

if DRY_RUN:
    print("\n⚠️  DRY RUN MODE - No changes will be made")

📋 Configuration loaded
   DRY_RUN: True
   TEST_LIMIT: None
   Output: account_enrichment_20260124_093007.xlsx

⚠️  DRY RUN MODE - No changes will be made


In [28]:
# === QUERY MIGRATED RECORDS ===

print("\n" + "=" * 80)
print("QUERYING MIGRATED RECORDS")
print("=" * 80)

# Build list of ES fields to query from the mapping
es_fields_needed = list(set(ENRICHMENT_MAPPING.values()))
es_fields_needed.extend(["Id", "BBF_New_Id__c", "Name"])  # Always need these

es_fields_str = ", ".join(es_fields_needed)

es_query = f"""
SELECT {es_fields_str}
FROM Account
WHERE BBF_New_Id__c != null
"""

if TEST_LIMIT:
    es_query += f" LIMIT {TEST_LIMIT}"

print(f"\n📌 Querying ES Account...")
print(f"   Fields: {len(es_fields_needed)}")
es_result = es_sf.query_all(es_query)
es_records = es_result["records"]
print(f"   Found {len(es_records)} migrated Account records")

# Build lookup by BBF_New_Id__c
es_lookup = {r["BBF_New_Id__c"]: r for r in es_records}

# Build list of BBF fields to query
bbf_fields_needed = list(ENRICHMENT_MAPPING.keys())
bbf_fields_needed.extend(["Id", "ES_Legacy_ID__c", "Name"])

bbf_fields_str = ", ".join(bbf_fields_needed)

# Query BBF Account current values
bbf_ids = list(es_lookup.keys())
bbf_records = []

print(f"\n📌 Querying BBF Account...")
print(f"   Fields: {len(bbf_fields_needed)}")
chunk_size = 200
for i in range(0, len(bbf_ids), chunk_size):
    chunk = bbf_ids[i : i + chunk_size]
    ids_str = "','".join(chunk)

    bbf_query = f"""
    SELECT {bbf_fields_str}
    FROM Account
    WHERE Id IN ('{ids_str}')
    """

    result = bbf_sf.query_all(bbf_query)
    bbf_records.extend(result["records"])

print(f"   Found {len(bbf_records)} Account records in BBF")


QUERYING MIGRATED RECORDS

📌 Querying ES Account...
   Fields: 53
   Found 2257 migrated Account records

📌 Querying BBF Account...
   Fields: 56
   Found 2257 Account records in BBF


In [29]:
# === BUILD ENRICHMENT UPDATES (Using Mapping Excel) ===

print("\n" + "=" * 80)
print("BUILDING ENRICHMENT UPDATES (From Excel Mapping)")
print("=" * 80)

updates = []
update_details = []  # For Excel output
field_stats = {
    field: {"enriched": 0, "already_set": 0, "no_source": 0}
    for field in ENRICHMENT_MAPPING
}

for bbf_rec in bbf_records:
    bbf_id = bbf_rec["Id"]

    # Get corresponding ES record
    if bbf_id not in es_lookup:
        continue
    es_rec = es_lookup[bbf_id]

    update_rec = {"Id": bbf_id}
    rec_details = {"bbf_id": bbf_id, "name": bbf_rec.get("Name", "N/A"), "fields": []}

    # Process each field from the mapping
    for bbf_field, es_field in ENRICHMENT_MAPPING.items():
        # Get current values
        bbf_value = bbf_rec.get(bbf_field)
        es_value = es_rec.get(es_field)

        if bbf_value:
            field_stats[bbf_field]["already_set"] += 1
        elif es_value:
            # Check if this field has picklist translations in the mapping
            transformed = translate_picklist(mapping, bbf_field, es_value)

            update_rec[bbf_field] = transformed
            rec_details["fields"].append(
                {
                    "field": bbf_field,
                    "old": bbf_value,
                    "new": transformed,
                    "source": es_value,
                }
            )
            field_stats[bbf_field]["enriched"] += 1
        else:
            field_stats[bbf_field]["no_source"] += 1

    # Only add if there are fields to update
    if len(update_rec) > 1:  # More than just Id
        updates.append(update_rec)
        update_details.append(rec_details)

print(f"\n📊 Enrichment Analysis:")
print(f"   Total BBF records analyzed: {len(bbf_records)}")
print(f"   Records needing updates: {len(updates)}")
print(f"\n   Field Statistics (from mapping Excel):")
print(f"   {'Field':<40} | {'Enriched':>10} | {'Already Set':>12} | {'No Source':>10}")
print(f"   {'-'*80}")
for field, stats in field_stats.items():
    print(
        f"   {field:<40} | {stats['enriched']:>10} | {stats['already_set']:>12} | {stats['no_source']:>10}"
    )


BUILDING ENRICHMENT UPDATES (From Excel Mapping)

📊 Enrichment Analysis:
   Total BBF records analyzed: 2257
   Records needing updates: 2257

   Field Statistics (from mapping Excel):
   Field                                    |   Enriched |  Already Set |  No Source
   --------------------------------------------------------------------------------
   ParentId                                 |         46 |            0 |       2211
   BillingCountry                           |          2 |         2255 |          0
   BillingLatitude                          |       2254 |            0 |          3
   BillingLongitude                         |       2254 |            0 |          3
   BillingGeocodeAccuracy                   |       2254 |            0 |          3
   BillingAddress                           |          0 |         2257 |          0
   ShippingStreet                           |          0 |           93 |       2164
   ShippingCity                             |     

In [30]:
# === APPLY ENRICHMENT UPDATES ===

print("\n" + "=" * 80)
print("APPLYING ENRICHMENT UPDATES")
print("=" * 80)

if len(updates) == 0:
    print("\n⚠️  No updates to apply")
    update_results = []
elif DRY_RUN:
    print(f"\n🔍 DRY RUN - Would update {len(updates)} Account records")
    print("\nSample updates (first 5):")
    for i, detail in enumerate(update_details[:5], 1):
        print(f"\n{i}. {detail['name']} ({detail['bbf_id'][:15]}...)")
        for f in detail["fields"][:3]:
            print(f"   {f['field']}: {f['old']} -> {f['new']}")
    update_results = [{"success": True, "id": u["Id"]} for u in updates]  # Mock results
else:
    print(f"\n📌 Updating {len(updates)} Account records...")
    print("   (Using small batches due to CPQ triggers)")

    update_results = []
    batch_size = 10  # Small batches for Account due to triggers

    for i in range(0, len(updates), batch_size):
        batch = updates[i : i + batch_size]
        try:
            results = bbf_sf.bulk.Account.update(batch)
            update_results.extend(results)
        except Exception as e:
            print(f"   Error in batch {i//batch_size + 1}: {e}")

    success_count = sum(1 for r in update_results if r["success"])
    error_count = sum(1 for r in update_results if not r["success"])

    print(f"\n✅ Successfully updated: {success_count}")
    print(f"❌ Failed to update: {error_count}")


APPLYING ENRICHMENT UPDATES

🔍 DRY RUN - Would update 2257 Account records

Sample updates (first 5):

1. Lorain County ESC (001Ea00001LEmWo...)
   BillingLatitude: None -> 41.390559
   BillingLongitude: None -> -82.129981
   BillingGeocodeAccuracy: None -> Block

2. AT&T (001Ea00001LEmWn...)
   BillingLatitude: None -> 38.864018
   BillingLongitude: None -> -77.366661
   BillingGeocodeAccuracy: None -> Address

3. Expereo (001Ea00001LEmWU...)
   BillingLatitude: None -> 38.945974
   BillingLongitude: None -> -77.313572
   BillingGeocodeAccuracy: None -> Address

4. Switch (001Ea00001LEmWT...)
   BillingLatitude: None -> 36.229
   BillingLongitude: None -> -115.2607
   BillingGeocodeAccuracy: None -> Zip

5. Corewell Health (001Ea00001LEmVN...)
   BillingLatitude: None -> 38.7518
   BillingLongitude: None -> -77.4728
   BillingGeocodeAccuracy: None -> Zip


In [31]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()

# --- Sheet 1: Summary ---
ws1 = wb.active
ws1.title = "Summary"
ws1.append(["Account Enrichment Summary"])
ws1["A1"].font = Font(bold=True, size=14)
ws1.append([])
ws1.append(["Mapping File:", MAPPING_FILE])
ws1.append(["Run Type:", "DRY RUN" if DRY_RUN else "LIVE UPDATE"])
ws1.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws1.append(["Records Analyzed:", len(bbf_records)])
ws1.append(["Records Updated:", len(updates)])
ws1.append([])
ws1.append(["Field (from mapping)", "Enriched", "Already Set", "No Source"])
for field, stats in field_stats.items():
    ws1.append([field, stats["enriched"], stats["already_set"], stats["no_source"]])

# --- Sheet 2: Update Details ---
ws2 = wb.create_sheet("Update Details")
headers = ["BBF Account ID", "Name", "Field", "Old Value", "New Value", "ES Source"]
ws2.append(headers)

header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="4472C4")
for col, header in enumerate(headers, 1):
    cell = ws2.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill

for detail in update_details:
    for f in detail["fields"]:
        ws2.append(
            [
                detail["bbf_id"],
                detail["name"],
                f["field"],
                str(f["old"]) if f["old"] else "",
                str(f["new"]),
                str(f["source"]),
            ]
        )

# Adjust column widths
for col in ws2.columns:
    max_length = max(len(str(cell.value or "")) for cell in col)
    ws2.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws2.freeze_panes = "A2"

# --- Sheet 3: Mapping Reference ---
ws3 = wb.create_sheet("Mapping Reference")
ws3.append(["Field Mappings Used (from Excel)"])
ws3["A1"].font = Font(bold=True, size=12)
ws3.append([])
ws3.append(["BBF Field", "ES Field"])
for bbf_field, es_field in ENRICHMENT_MAPPING.items():
    ws3.append([bbf_field, es_field])

# Save
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")


CREATING EXCEL OUTPUT

✅ Excel output saved to: account_enrichment_20260124_093007.xlsx


In [32]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("ACCOUNT ENRICHMENT COMPLETE")
print("=" * 80)
print(f"\nMapping file used: {MAPPING_FILE}")
print(f"Records analyzed: {len(bbf_records)}")
print(f"Records {'would be ' if DRY_RUN else ''}updated: {len(updates)}")
print(f"Output file: {output_file}")

if DRY_RUN:
    print("\n⚠️  This was a DRY RUN - no changes were made")
    print("   Set DRY_RUN = False to apply updates")


ACCOUNT ENRICHMENT COMPLETE

Mapping file used: ES_Account_to_BBF_Account_mapping.xlsx
Records analyzed: 2257
Records would be updated: 2257
Output file: account_enrichment_20260124_093007.xlsx

⚠️  This was a DRY RUN - no changes were made
   Set DRY_RUN = False to apply updates
